# Django ORM

## Настройка БД. Django ORM. Django models

Note: Проверьте что у вас создался файл 'db.sqlite3'

In [ ]:
# tms_django_lessons/settings.py
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': BASE_DIR / 'db.sqlite3',
    }
}

## Модели

https://docs.djangoproject.com/en/3.2/topics/db/models/
https://djangodoc.ru/3.1/topics/db/models/

**Модель** - это единственный и окончательный источник информации о ваших данных. Он содержит основные поля и поведение хранимых вами данных. Обычно каждой модели соответствует одна таблица в базе данных.

Основы :
- Каждая модель - это класс Python, наследующий от django.db.models.Model .
- Каждый атрибут модели представляет поле базы данных.
- Наряду со всем этим Django предоставляет API для доступа к автоматически сгенерированной базе данных; см. Создание запросов .

In [ ]:
# articles/models.py
from django.db import models


class Article(models.Model):
    title = models.CharField(max_length=50)
    content = models.TextField()
    author_name = models.CharField(max_length=50)
    rating = models.IntegerField()

Класс выше будет соответствовать таблице вида:
```sql
create table articles_article
(
    id          integer     not null
        primary key autoincrement,
    title       varchar(50) not null,
    content     text        not null,
    author_name varchar(50) not null,
    rating      integer     not null
);
```

Note:
- название таблицы по умолчанию берется из названия приложения и названия таблицы переведенного из формата [PascalCase в camel_case](https://www.theserverside.com/answer/Pascal-case-vs-camel-case-Whats-the-difference)
  - имя менять можно https://docs.djangoproject.com/en/3.2/ref/models/options/#db-table
- id - колонка по умолчанию добавляется в каждую таблицу, которая унаследована от models.Model
  - колонку менять можно https://marcolcl.medium.com/custom-django-model-field-based-on-default-primary-key-d5d13dd61a1c

## makemigrations and migrate

- После создания класса, таблица не создастся в базе не появилась:(
- Чтобы исправить ситуацию нам потребуется запустить 2 команды в консоли:
```shell
python manage.py makemigrations
python manage.py migrate
```

#### makemigrations
- https://docs.djangoproject.com/en/3.2/ref/django-admin/#django-admin-makemigrations
- Сгенерирует файл в папке `articles/migrations/0001_initial.py`

#### migrate
- https://docs.djangoproject.com/en/3.2/ref/django-admin/#migrate
- Применит все файлы которые находятся в папках `migrations` из всех приложений описанных в [INSTALLED_APPS](https://docs.djangoproject.com/en/3.2/ref/settings/#installed-apps)

Note: по умолчанию django предостваляет и свои файлы миграций, потому в базе данных будет создана больше чем одна таблица

## Задание 1. 
Добавить колонку `topic varchar(50) null `, создать миграции и применить их

Примечание: в файле `tms_django_lessons/settings.py` обязательно должно быть добавленно приложение `articles` в INSTALLED_APPS

## django shell

- PyCharm Pro по умолчанию предоставляет консоль для работы с django
- Если у вас не PyCharm Pro то вы можете запустить в консоли команду 
```shell
python manage.py shell
```

## Создание записей в БД используя ORM

Три способа сохранить данные в бд:

In [ ]:
# 1
from articles.models import Article

article = Article(
    title='Olympic Games 2021', 
    content='not bad games', 
    author_name='winner',
    rating=5,              
)
article.save()

In [ ]:
# 2 
from articles.models import Article

article = Article()
article.title = 'Belarus 2021'
article.content = 'not bad country'
article.author_name = 'bulba'
article.rating = 8
article.save()

In [ ]:
# 3
from articles.models import Article

article = Article.objects.create(
    title='TeachMeSkills', 
    content='not bad school', 
    author_name='trainer',
    rating=10, # :)
)

## Получение записей из БД
```python
from articles.models import Article

Article.objects.filter()  # получение всех записей из БД

Article.objects.filter(title='Belarus 2021')  # получение всех записей из БД  подходящих по условию

Article.objects.get(id=2)  # получение одной конкретно записи

Article.objects.filter().first()  # получение первого элемента
```

## Задание 2
- Создать 3 статьи. 
- Получить статьи с именем News. 
- Получить статью с id 1.

## Удаление записей
```python
from articles.models import Article

customer = Article.objects.get(id=5)
customer.delete()  # удаление одной записи

Article.objects.filter(title='News').delete()  # Удаление множество записей
```

## Задание 3

Удалить трех пользователей

## Обновление записей в бд

Два способа обновления:

In [ ]:
# 1
article = Article.objects.get(id=5)
article.content = 'Wow cool!'
article.save()

In [ ]:
# 2
Article.objects.filter(title='Belarus 2021').update(title='Belarus Forever')

## Задание 4
Обновить 3 пользователя

## Задание 5 (вместе)

Добавить возможность создания articles через сайт

### Добавим класс для формы создания article

Note: обычно название формы создается по шаблону МодельДействиеРодитель


ArticleCreateForm:
- Article == Модель
- Create == Действие
- Form == Родитель

In [ ]:
# articles/forms.py

from django import forms


class ArticleCreateForm(forms.Form):
    title = forms.CharField(max_length=50)
    content = forms.CharField()
    author_name = forms.CharField(max_length=50)
    rating = forms.IntegerField()
    topic = forms.CharField(max_length=50, required=False)

### Добавим html файл `add_article.html` для создания article

Note: нам не важно как будет выглядить {{ form }} переданный в html файл

templates/add_article.html
```html
{% extends "base.html" %}

{% block title %}Add article{% endblock %}

{% block content %}
    <h1>
        Article creation
    </h1>
    <form action="" method="post">
        {% csrf_token %}
        {{ form }}
        <input type="submit" name="Добавить статью">
    </form>
{% endblock %}
```

## Создать функцию add_article

In [ ]:
# articles/views.py

# ...

from articles.models import Article
from articles.forms import ArticleCreateForm

# ...

def add_article(request):
    if request.method == 'POST':
        form = ArticleCreateForm(request.POST)
        if form.is_valid():
            Article.objects.create(
                title=form.cleaned_data['title'],
                content=form.cleaned_data['content'],
                author_name=form.cleaned_data['author_name'],
                rating=form.cleaned_data['rating'],
                topic=form.cleaned_data.get('topic'),
            )
    else:
        form = ArticleCreateForm()
    return render(
        request,
        'add_article.html',
        context={'form': form}
    )


## Cоздать новый url `add-article/`

In [ ]:
# tms_django_lessons/urls.py

# ...

from articles.views import index, home_page, get_article_page, add_article

urlpatterns = [
    # ...
    path('add-article/', add_article),
]

## Задание 6
Протестировать все, что добавили

## Перенаправление. 3 варианта


```python
from django.shortcuts import redirect

# 1
return redirect(thanks_page)  # на конкретную view
# 2 
return redirect('thanks_page_name')  # на имя определенное в urls.py
# 3 
return redirect('/thanks/') # на определенный урл
```

## Задание 7 (HW)

- После создания пользователя -  перенаправлять на домашнюю страницу. 
- Добавить возможность переход на форму создания с домашней страницы.

## Задание 8

Отображать все Article на домашней страницe

Note: Удалим ненужную форму WriteLineForm

## Перепишем функцию `home_page` чтобы она возвращала все Article
```python
# articles/views.py
def home_page(request):
    articles = Article.objects.all()
    return render(
        request,
        'home.html',
        context={"articles": articles},
    )
```
## Перепишем файл `home.html` чтобы он мог отображать все Article
templates/home.html
```html
{% extends "base.html" %}

{% block title %}Home Page{% endblock %}

{% block content %}
    <h1>
        Hello user
    </h1>
    {% for article in articles %}
        {{ article }} <br>
    {% endfor %}
{% endblock %}
```